# Filter data used in vacancy model based on TCI parcel numbers

Parcels surveyed in Summer 2015 so all data pulled should come from before that. Goal of script/notebook is to filter datasets by the parcel numbers in the TCI survey, although we will filter again based on existence of structure.

In [1]:
import pandas as pd
import csv
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import datetime as dt
%matplotlib inline

path = "Z:/largetransfer/luc/carter"
print(path)

Z:/largetransfer/luc/carter


## Cleaning file: "Cleveland_Final_Results_Table_FOR_DISTRIBUTION_20151111.xlsx"

Using the "Survey Category" feature we encode it "Occupied Structure" and "Vacant Structure" as 0 or 1 respectively. However if the parcel has any other status apart from the above two status then we encode it as -1.

We then create two new variable "vacant" and "parcels" for the encoding the vacancy status and parsing the parcel IDs.


**Note**: File written to `tci_1_0.csv`

In [2]:
tci = pd.read_excel(path+'/data/inspection_data/Cleveland_Final_Results_Table_FOR_DISTRIBUTION_20151111.xlsx', encoding="ISO-8859-1") 

# def get_vacant(x):
#     if x == 'Occupied Structure':
#         return 0
#     elif x == 'Vacant Structure':
#         return 1
#     else: 
#         return -1
    
# tci['vacant'] = tci['Survey Category'].apply(get_vacant)
# tci['parcel'] = tci.PIN.apply(lambda x: x[0:3]+'-'+x[3:5]+'-'+x[5:])

# tci[(tci.USE_CLASS=='R') & (tci.vacant>-1)].to_csv(path+'/data/model_data/tci_1_0.csv', index=False)

In [5]:
tci['Survey Category'].unique()

array(['Occupied Structure', 'Vacant Lot', 'Park', 'Vacant Structure',
       'With Adjacent', 'Parking Lot', 'Not Surveyed'], dtype=object)

#### Number of vacant parcels

In [16]:
sum(tci[tci.vacant>0].vacant)
#  or 
#tci["Survey Category"].value_counts()

12179

#### Number of Residential parcels

In [5]:
tci[(tci.USE_CLASS=='R')].shape

(131522, 33)

#### Grabing the parcels IDs for residential housings that are either occupied or vacant

In [7]:
tci = pd.read_csv(path+'/data/model_data/tci_1_0.csv', parse_dates=['Date'], dtype={'Ward':object,'PIN':str})
ppns = set(tci[(tci.USE_CLASS=='R') & (tci.vacant>-1)].parcel)     #parcel ids of either occupied or vacant residentials

### Demographic data

Here we merge the Cleveland parcel census data with the sociodemographic data on ** *`census tract`* **

Select a handful of columnns of the resulting dataframe and saving it to ** *`demographic.csv`* **

In [ ]:
tracts = pd.read_csv(path+'/data/original_data/clv_par_census.csv')
demo = pd.read_csv(path+'/data/original_data/sociodemographic_Data.csv')

tracts = pd.merge(tracts, demo, left_on='NAME10', right_on='Census Tract', how='left')
cols = [0,1,5,7,9,11,13,14,15,16,17,18,20,22,24,26,28,30,32,34,36,38,40]
tracts.iloc[:,cols].to_csv(path+'/data/clean_data/demographic.csv',index=False)

## Property characteristics

In [ ]:
df = pd.read_csv(path+'/data/original_data/main_prop.csv')

In [2]:
import simpledbf

In [5]:
dbf = simpledbf.Dbf5(path+'/data/original_data/parcel0611_lookup2010.dbf').to_dataframe()

In [11]:
dbf[dbf.CITY=='CLEVELAND'].groupby('PARCEL').first().shape

(178548, 14)

####  Collecting parcels from "main properties" which matches the parcel IDs
####  in Cleveland_Final_Results_Table_FOR_DISTRIBUTION_20151111

This file is written to ** * `main_prop13.csv`* **

In [ ]:
# infile = path+'/data/original_data/main_prop.csv'
# outfile = path+'/data/clean_data/main_prop13.csv'

# with open(infile, 'r', encoding="ISO-8859-1") as fin, open(outfile, 'w') as fout:
#     write_to = csv.writer(fout, lineterminator='\n')
#     header = next(csv.reader(fin))
#     write_to.writerow(header)
#     for row in csv.reader(fin):
#         if row[0] in ppns:
#             write_to.writerow(row)

##### In this cell, we subset the **`main_prop13.csv`**  for data that corresponds to parcels in **tci_1_0.csv**
#### Next we collect a the data for the tax year 2013 and 2014, merge these and write it out to **`main_prop_filtered.csv`**

In [ ]:
# commented code only needs to be run once

main = pd.read_csv(path+'/data/clean_data/main_prop13.csv', dtype=object)

main = main.drop_duplicates()
main = main[main.parcel.isin(ppns)]

main14 = main[main.taxyr=='2014'].groupby('parcel').first().reset_index()
main13 = main[main.taxyr=='2013'].groupby('parcel').first().reset_index()

print(main13.shape, main14.shape)

In [ ]:
main13 = main13.set_index('parcel')
main13 = main13.rename(columns={'condition':'condition13'})
main14 = main14.rename(columns={'condition':'condition14'})
pd.merge(main14, main13[['condition13']], how='left', left_on='parcel', right_index=True)\
    .to_csv(path+'/data/clean_data/main_prop_filtered.csv')

## Residential characteristics 
Filename: ```res2014.csv```

We collect part of this data set which has parcel IDs contained in **`tci_1_0.csv:filtered cleveland final result for distribution 20151111 `**

In [10]:
res.groupby('parcel').count().shape

(425302, 46)

In [6]:
# first part only needs to be run once

res = pd.read_csv(path+'/data/original_data/res/res2014.csv')
# res = res[res.parcel.isin(ppns)]
# res.to_csv(path+'/data/clean_data/res.csv', index=False)

# res = pd.read_csv(path+'/data/clean_data/res.csv')

## Tax bill
Filename: ```dec14_tci.csv```

In [3]:
tb = pd.read_csv(path+'/data/clean_data/taxbill_may15.csv')

IOError: File /data/clean_data/taxbill_may15.csv does not exist

#### We collect parcel data in the ` taxbill/may15.csv` that matches the parcels IDs in `tci_1_0.csv: filtered Cleveland final(2015111)` 
#### We then write this file to `taxbill_may15.csv`

In [ ]:
##only needs to be run once to clean taxbill data

infile = path+'/data/original_data/taxbill/may15.csv'
outfile = path+'/data/clean_data/taxbill_may15.csv'

with open(infile, 'r') as fin, open(outfile, 'w') as fout:
    write_to = csv.writer(fout, lineterminator='\n')
    header = next(csv.reader(fin))
    write_to.writerow(header)
    for row in csv.reader(fin):
        if row[5] in ppns:
            write_to.writerow(row)

#### Here we collect parcel data from the `taxbill/sep14.csv` and write to `taxbill_sep14.csv` data for parcels that correspond 
#### to the one we have in `tax_1_0.csv`

In [ ]:
##only needs to be run once to clean taxbill data

infile = path+'/data/original_data/taxbill/sep14.csv'
outfile = path+'/data/clean_data/taxbill_sep14.csv'

with open(infile, 'r') as fin, open(outfile, 'w') as fout:
    write_to = csv.writer(fout, lineterminator='\n')
    header = next(csv.reader(fin))
    write_to.writerow(header)
    for row in csv.reader(fin):
        if row[5] in ppns:
            write_to.writerow(row)

## County land bank
Filename: ```count_land_bank.csv```

##### As usual we collect data of parcels that corresponds to parcels IDs in `tci_1_.csv`from `count_land_bank.csv` to `county_lb.csv`

In [ ]:
lb = pd.read_csv(path+'/data/original_data/count_land_bank.csv', parse_dates=[3,4])

lb = lb[lb['parcel'].isin(ppns)]
# lb = lb[lb['acq_dt']<np.datetime64('2015-06-01')]

lb.to_csv(path+'/data/clean_data/county_lb.csv', index=False)

## Foreclosure filings

#### We collect data of parcel that corresponds to parcel IDs in `tci_1_0.csv` from `foreclosure_filings2006_beyond.csv` to  
#### `foreclosure_filings2.csv`

In [8]:
fc = pd.read_csv(path+'/data/original_data/foreclosure_filings2006_beyond.csv', parse_dates = [2])

fc = fc[fc['parcel'].isin(ppns)]
# fc = fc[fc['filedate']<np.datetime64('2015-06-01')]

fc.to_csv(path+'/data/clean_data/foreclosure_filings2.csv',index=False)

In [10]:
fc.parcel_address

1148           11516 EDGEWATER DRIVE
1149           11436 EDGEWATER DRIVE
1150           11315 EDGEWATER DRIVE
1151           11425 EDGEWATER DRIVE
1152           11439 EDGEWATER DRIVE
1153               11312 LAKE AVENUE
1154           11015 EDGEWATER DRIVE
1155               11110 LAKE AVENUE
1156               11018 LAKE AVENUE
1157             10710 EDGEWATER DR.
1158             10710 EDGEWATER DR.
1159           10700 EDGEWATER DRIVE
1160                  2402 CLOFAX RD
1161           10801 EDGEWATER DRIVE
1162               10900 LAKE AVENUE
1163               10806 LAKE AVENUE
1164               10416 LAKE AVENUE
1165               10001 CLIFF DRIVE
1166               10001 CLIFF DRIVE
1167               10001 CLIFF DRIVE
1168               10001 CLIFF DRIVE
1169               10001 CLIFF DRIVE
1170                  9995 CLIFF DR.
1171                  9995 CLIFF DR.
1172           10217 EDGEWATER DRIVE
1173           10001 EDGEWATER DRIVE
1174               10010 LAKE AVENUE
1

In [ ]:
# fc2 = pd.read_csv(path+'/data/original_data/foreclosure_filings2006_dec2014.csv', parse_dates = [2])

## Sheriff auction
Filename: ```shf_aution_mar2000_dec2014.csv```

#### We collect data that corresponds to parcel IDs in `tci_1_0.csv` from `shf_aution_mar2000_dec2014.csv` and write this to 
#### `sheriff_auction.csv`. Next we subset further for apporiate dates

In [ ]:
sa = pd.read_csv(path+'/data/original_data/shf_aution_mar2000_dec2014.csv', parse_dates=[2], encoding="ISO-8859-1")

sa = sa[sa.parcel.isin(ppns)]
# sa = sa[sa.salesdt<np.datetime64('2015-06-01')]

sa.to_csv(path+'/data/clean_data/sheriff_auction.csv', index=False)

## Transfers
Filename: ```transfers2000_2014.csv```


#### We collect data that corresponds to parcel IDs in `tci_1_0.csv` from `transfers2000_2014.csv` and write this to 
#### `transfers.csv`. Next we subset further for apporiate dates

In [ ]:
infile = path+'/data/original_data/transfers2000_2014.csv'
outfile = path+'/data/clean_data/transfers.csv'

with open(infile, 'r') as fin, open(outfile, 'w') as fout:
    write_to = csv.writer(fout, lineterminator='\n')
    header = next(csv.reader(fin))
    write_to.writerow(header)
    for row in csv.reader(fin):
        if row[5] in ppns:
            write_to.writerow(row)

In [ ]:
max(tf.mdate)

In [ ]:
# tf = pd.read_csv(path+'/data/clean_data/transfers.csv',parse_dates=[8], dtype='str')
# tf = tf[tf.mdate<np.datetime64('2015-06-01')]

# tf.to_csv(path+'/data/clean_data/transfers.csv',index=False)

## Armslength sales
Filename: ```armslengthsales2006_2014.csv```

#### We collect data that corresponds to parcel IDs in `tci_1_0.csv` from `armslengthsales2006_beyond.csv` and write this to 
#### `armslength.csv`. Next we subset further for apporiate dates

In [ ]:
al = pd.read_csv(path+'/data/original_data/armslengthsales2006_beyond.csv', dtype=str)

In [ ]:
print al.columns.values

In [ ]:
al = pd.read_csv(path+'/data/original_data/armslengthsales2006_beyond.csv', dtype=str)
al = al[al.PROPERTY_NUMBER.isin(ppns)]
al.to_csv(path+'/data/clean_data/armslength.csv',index=False)

# al = pd.read_csv(path+'/data/clean_data/armslength.csv', parse_dates=['mdate'])

## Violations
Filename: ```violate_cle.csv```

#### We collect data that corresponds to parcel IDs in `tci_1_0.csv` from `violate_cle.csv` and write this to 
#### `violations.csv`. Next we subset further for apporiate dates


In [ ]:
infile = path+'/data/original_data/violate_cle.csv'
outfile = path+'/data/clean_data/violations.csv'

v = pd.read_csv(infile, dtype=str)
v = v[v.parcel.isin(ppns)]
# v = v[(v.v_file_date < np.datetime64('2015-06-01')) & (v.v_file_date > np.datetime64('2006-06-01'))]

v.to_csv(path+'/data/clean_data/violations.csv',index=False)

## Complaints
Filename: ```complaint_cle.csv```

#### We collect data that corresponds to parcel IDs in `tci_1_0.csv` from `complaint_cle.csv` and write this to 
#### `complaints.csv`. Next we subset further for apporiate dates

In [2]:
c = pd.read_csv(path+'/data/original_data/complaint_cle.csv', dtype=str)

c = c[c.parcel.isin(ppns)]
# c = c[(c.c_file_date < np.datetime64('2015-06-01')) & (c.c_file_date > np.datetime64('2006-06-01'))]

c.to_csv(path+'/data/clean_data/complaints.csv')

## Postal data
Filenames: ```pv201302.csv, pv201304.csv, pv201308.csv, pv201312.csv, pv201402.csv```

#### For each of the 31 files in "postal", we attach and time stamp to the data. Then we combine all those data sets into one giant data set. After which we collect the observations( parcels) that corresponds to the parcel IDs in `tci_1_0.csv` and then write it to `postal_vacancy.csv`

In [ ]:
len(set(p.PARCEL))

In [ ]:
p = pd.DataFrame()
for pv in os.listdir(path+'/data/original_data/postal/'):
    pos = pd.read_csv(path+'/data/original_data/postal/' + pv)
    pos['date'] = dt.datetime(int(pv[2:6]), int(pv[6:8]), 1)
    p = p.append(pos)
p = p[p.PARCEL.isin(ppns)]
p.to_csv(path+'/data/clean_data/postal_vacancy.csv')